# About this Dataset <a class="anchor" id="about"></a>

  The dataset was created by The Criteo AI Lab .The dataset consists of 13M rows, each one representing a user with 12 features, a treatment indicator and 2 binary labels (visits and conversions). Positive labels mean the user visited/converted on the advertiser website during the test period (2 weeks). The global treatment ratio is 84.6%. It is usual that advertisers keep only a small control population as it costs them in potential revenue.  
  
Following is a detailed description of the features:  
  
- f0, f1, f2, f3, f4, f5, f6, f7, f8, f9, f10, f11: feature values (dense, float)
- treatment: treatment group (1 = treated, 0 = control)
- conversion: whether a conversion occured for this user (binary, label)
- visit: whether a visit occured for this user (binary, label)
- exposure: treatment effect, whether the user has been effectively exposed (binary)

In [1]:
!pip install causalml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 24.5 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.7/278.7 kB 16.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 71.1 MB/s eta 0:00:00
  Created wheel for causalml: filename=causalml-0.15.2-cp37-cp37m-linux_x86_64.whl size=6428256 sha256=20f90fac5a91c2f5bab4907de6d2d4029f6abcf8d1c44bf1bb5798983cfbd497
  Stored in directory: /root/.cache/pip/wheels/77/7b/df/ae934d53609fe0cd61502f11c72ed039222b98e08cdebe2a06
  Created wheel for pydotplus: filename=pydotplus-2.0.2-py3-none-any.whl size=24575 sha256=8511effd688475ac6aebc168331dc4d8aca034ae8c1127f85164829352ba6c57
  Stored in directory: /root/.cache/pip/wheels/1e/7b/04/7387cf6cc9e48b4a96e361b0be812f0708b394b821bf8c9c50
Successfully built causalml pydotplus


In [2]:
!pip install scikit-uplift

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 2.9 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklift.metrics import uplift_at_k, uplift_auc_score, qini_auc_score, weighted_average_uplift
from sklift.viz import plot_uplift_preds
from sklift.models import SoloModel, TwoModels
from sklearn.base import clone
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from causalml.inference.tree import UpliftRandomForestClassifier, UpliftTreeClassifier
from causalml.inference.tree import uplift_tree_string, uplift_tree_plot
from IPython.display import Image

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import shap

def prepare_data(df, feature_cols, treatment_col='treatment', target_col='conversion', 
                 test_size=0.2, random_state=42):
    """
    Prepares data for uplift modeling with binary treatment/control setup
    """
    # Split data into features, treatment and target
    X = df[feature_cols]
    t = df[treatment_col]
    y = df[target_col]

    # Train-test split with stratification on treatment
    X_train, X_test, t_train, t_test, y_train, y_test = train_test_split(
        X, t, y, test_size=test_size, random_state=random_state, stratify=t
    )

    return {
        'X_train': X_train, 'X_test': X_test,
        't_train': t_train, 't_test': t_test,
        'y_train': y_train, 'y_test': y_test
    }

# Example usage
feature_cols = [f'f{i}' for i in range(12)]  # Update with your actual feature names
df = pd.read_csv('../input/uplift-modeling/criteo-uplift-v2.1.csv')
data = prepare_data(df, feature_cols)

# Access prepared data
X_train = data['X_train']
X_test = data['X_test']
t_train = data['t_train']
t_test = data['t_test']
y_train = data['y_train']
y_test = data['y_test']


In [8]:
from causalml.inference.meta import BaseSRegressor, BaseTRegressor, XGBTRegressor
from causalml.inference.tree import UpliftRandomForestClassifier
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor
# Define treatment actions
actions = feature_cols

# Initialize and train T-Learner

learner_t=XGBTRegressor(random_state=42)

learner_tresult=learner_t.fit_predict(X_train,t_train,y_train)
print(learner_tresult)

[[-1.09978835e-04]
 [-2.85150709e-05]
 [-4.72744927e-04]
 ...
 [ 2.33899249e-04]
 [-6.25580578e-05]
 [-2.85150709e-05]]


In [18]:
from causalml.inference.meta import MLPTRegressor

nn = MLPTRegressor(hidden_layer_sizes=(35, 25, 10, 5),
                 learning_rate_init=.01,
                 early_stopping=True,
                 random_state=1)

In [60]:
import pickle
# Save the entire model (including its weights) to a checkpoint file
checkpoint_filename = "synapses_checkpint.pkt"
with open(checkpoint_filename, "wb") as f:
    pickle.dump(nn, f)

print(f"Model checkpoint saved to {checkpoint_filename}")


Model checkpoint saved to synapses_checkpint.pkt


In [20]:
# ITE
nn_ite = nn.fit_predict(X_train, t_train, y_train)

In [51]:
nn_ite.shape

(11183673, 1)

In [24]:
# Feature importance using permutation
result = nn.get_importance(X=X_train, tau=nn_ite, method='permutation', features=actions, random_state=42)
result

{1: f4     1.022868
 f9     0.580809
 f3     0.088367
 f6     0.026656
 f5     0.019723
 f2     0.013324
 f10    0.013275
 f11    0.012541
 f0     0.005272
 f7     0.002762
 f8     0.001566
 f1     0.000378
 dtype: float64}

In [56]:
feature_results = result[1]
feature_results

f4     1.022868
f9     0.580809
f3     0.088367
f6     0.026656
f5     0.019723
f2     0.013324
f10    0.013275
f11    0.012541
f0     0.005272
f7     0.002762
f8     0.001566
f1     0.000378
dtype: float64

In [58]:
feature_results = feature_results/feature_results.sum() * 100
feature_results

f4     57.222065
f9     32.492037
f3      4.943517
f6      1.491185
f5      1.103343
f2      0.745393
f10     0.742628
f11     0.701595
f0      0.294946
f7      0.154502
f8      0.087618
f1      0.021171
dtype: float64